# Drug DB Tutorial

This tutorial will go through how to
#1) load a networkx graph into the db_network class object
#2) filter the database to generate a sub-set for a classification problem
#3) load and calculate a dictionary that store the properties of drug nodes
#4) train and evaluate a multi-label random forest model

## Step 1) Loading a networkx graph into a db_network object

In [1]:
import db_network as db     ### import db_network

mgraph = db.unpickle('DB_multigraph.obj')    ### Lets unpickle this sample graph
db_net = db.db_network(mgraph)      ## loads the mgraph networkx object into the db_network object

In [2]:
db_net.get_n_neighbors('Q9Y2H1',1)  ### Get the nearest 1-hop neighbor of node Q9Y2H1

['Q9Y2H1',
 'CHEMBL1725279',
 'CHEMBL509032',
 'CHEMBL1908397',
 'CHEMBL1721885',
 'CHEMBL388978',
 'CHEMBL603469',
 'CHEMBL574738',
 'CHEMBL607707',
 'CHEMBL572878',
 'CHEMBL535']

# Step 2) filter the database to generate a sub-set for a classification problem

In [3]:
edge_filter_dict={'weight': lambda x : x > 3.0}   ## only include drug-protein interactions with pChembl > 3.0
is_multilabel = True            ### flag for Multilabel classification
sel_nodes_list = ['P11362','P12931','P00533']    ### Select protein nodes
null_nodes_list = ['P24941','P00734']      ## Null node set (optional). Null proteins represent proteins that specifically don't bind the sel_node _list
X_mol,y_labels = db.get_class_set(db_net,edge_filter_dict,sel_nodes_list,null_nodes_list,is_multilabel)

In [10]:
len(X_mol), len(y_labels)   

(19277, 19277)

# Step 3) load and calculate properties of drug nodes

In [4]:
smiles_dict = db.unpickle('smiles_dict.obj')   ### dictionary of smiles for each CHEMBL molecule

In [5]:
prop_class = db.db_node_dict(smiles_dict) ## converts property dict to db_node_dict object

In [6]:
from rdkit import DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import MACCSkeys
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Descriptors3D
from rdkit.Chem import AllChem
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [7]:
#### Generating MACCS fingerprint of each molecule from the smiles, use these as features

X_hash_vals = []
for drug in X_mol:
    dd = prop_class.prop_dict[drug]['smiles']
    m = Chem.MolFromSmiles(dd)
    AllChem.Compute2DCoords(m)
    X_hash_vals.append(map(int,DataStructs.BitVectToText(MACCSkeys.GenMACCSKeys(m))))

In [8]:
### perform one-hot encoding on the interacting proteins
mlt = MultiLabelBinarizer().fit(y_labels)
pred_classes = mlt.transform(y_labels)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_hash_vals,pred_classes)  ## train/test split 75/25

In [10]:
labels = mlt.classes_   ## get labels of one-hot encoding

In [11]:
### calculate other properties from the smiles using the rd_mol helper object
prop_list = db.rd_mol(dd).rdkit_base_info['2D_Props']


X_prop_vals = []
for drug in X_mol:
    dd = prop_class.prop_dict[drug]['smiles']
    rd_mol = db.rd_mol(dd)
    rd_mol.get_2d_properties(prop_list)
    X_prop_vals.append([rd_mol.prop_2d_collector[prop] for prop in prop_list])


C:\Users\jkatigba\Anaconda2\lib\site-packages\numpy\core\fromnumeric.py:1424: RuntimeWarning: overflow encountered in reduce
  return asanyarray(a).trace(offset=offset, axis1=axis1, axis2=axis2, dtype=dtype, out=out)
C:\Users\jkatigba\Anaconda2\lib\site-packages\rdkit\Chem\Graphs.py:48: RuntimeWarning: invalid value encountered in multiply
  Bn = An - res[n] * I


# Step 4) train and evaluate a multi-label random forest model


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score, confusion_matrix

In [13]:
### training a random forest model on the MACCS fingerprint
forest = RandomForestClassifier(n_estimators=100, criterion = 'entropy', random_state = 123)
forest.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

In [14]:
## prediction
y_pred = forest.predict(X_test)

In [15]:
### printing out the results
result_dict = {}
for lx,label in enumerate(labels):
    result_dict[label]={}
    result_dict[label]['accuracy']=accuracy_score(y_test[:,lx],y_pred[:,lx])
    result_dict[label]['recall']=recall_score(y_test[:,lx],y_pred[:,lx])
    result_dict[label]['precision']=precision_score(y_test[:,lx],y_pred[:,lx])
    result_dict[label]['f1_score']=f1_score(y_test[:,lx],y_pred[:,lx])
    result_dict[label]['auroc']=roc_auc_score(y_test[:,lx],y_pred[:,lx])


In [16]:
### Multi-label random forest performs well in classifying drugs by their protein partner!
for label in labels:
    print(label,'accuracy:'+str(round(result_dict[label]['accuracy'],3)),'recall:'+str(round(result_dict[label]['recall'],3)),'precision:'+str(round(result_dict[label]['precision'],3)),'f1_score:'+str(round(result_dict[label]['f1_score'],3)),'auroc:'+str(round(result_dict[label]['auroc'],3)))

('P00533', 'accuracy:0.954', 'recall:0.889', 'precision:0.954', 'f1_score:0.921', 'auroc:0.936')
('P11362', 'accuracy:0.964', 'recall:0.75', 'precision:0.947', 'f1_score:0.837', 'auroc:0.872')
('P12931', 'accuracy:0.951', 'recall:0.76', 'precision:0.938', 'f1_score:0.84', 'auroc:0.875')
('null', 'accuracy:0.959', 'recall:0.95', 'precision:0.961', 'f1_score:0.955', 'auroc:0.958')
